In [97]:
from collections import deque
from typing import List, Tuple, Set
from Lab_9 import can_partition

In [98]:

class NоnDeterministicTuringMachineWrapper:
    def __init__(self):
        self.machine = NonDeterministicTuringMachine()  # Ініціалізація класу
        self.output = []

    def run(self, input_str: str) -> bool:
        # Виклик реального методу без додаткового виводу
        result = self.machine.run(input_str)
        self.output = ["Результат виконання збережено"]
        return result

In [99]:


class Tape:
    def __init__(self, input_str: str):
        self.tape = list(input_str)
        self.position = 0
        self.min_pos = 0
        self.max_pos = len(self.tape) - 1

    def read(self) -> str:
        if self.position < 0 or self.position >= len(self.tape):
            return ' '
        return self.tape[self.position]

    def write(self, symbol: str):
        while self.position >= len(self.tape):
            self.tape.append(' ')
        if self.position < 0:
            self.tape = [' '] * (-self.position) + self.tape
            self.position = 0
        self.tape[self.position] = symbol
        self.min_pos = min(self.min_pos, self.position)
        self.max_pos = max(self.max_pos, self.position)

    def move(self, direction: str):
        if direction == 'L':
            self.position -= 1
        elif direction == 'R':
            self.position += 1

    def __str__(self) -> str:
        return ''.join(self.tape[self.min_pos:self.max_pos + 1]) + f" (head at {self.position})"

    def copy(self):
        new_tape = Tape('')
        new_tape.tape = self.tape[:]
        new_tape.position = self.position
        new_tape.min_pos = self.min_pos
        new_tape.max_pos = self.max_pos
        return new_tape

class NonDeterministicTuringMachine:
    def __init__(self):
        self.states = {
            'q0', 'q_find_number', 'q_count_zeros', 'q_assign',
            'q_skip_number', 'q_compare', 'q_accept', 'q_reject'
        }
        self.tape = None
        self.state = 'q0'
        self.sigma = {'0', '1'}
        self.gamma = {'0', '1', ' ', 'A', 'B'}
        self.seen_configurations = set()

    def validate_input(self, input_str: str) -> bool:
        if not input_str:
            return True
        i = 0
        while i < len(input_str):
            if input_str[i] != '1':
                return False
            i += 1
            
            if i >= len(input_str) or input_str[i] != '0':
                return False
            while i < len(input_str) and input_str[i] == '0':
                i += 1
        return True

    def initialize_tape(self, input_str: str):
        if not all(c in {'0', '1'} for c in input_str):
            raise ValueError("Вхід має складатися лише з 0 і 1")
        if not self.validate_input(input_str):
            raise ValueError("Вхід має відповідати формату 10^i, де i >= 1")
        self.tape = Tape(input_str)
        self.state = 'q0'
        self.seen_configurations = set()

    def step(self, state: str, tape: Tape) -> List[Tuple[str, Tape]]:
        symbol = tape.read()
        next_configs = []

        if state == 'q0':
            if symbol == '1':
                new_tape = tape.copy()
                new_tape.position -= 1
                next_configs.append(('q_assign', new_tape))
            elif symbol == ' ':
                new_tape = tape.copy()
                new_tape.position = 0
                next_configs.append(('q_compare', new_tape))

        elif state == 'q_assign':
            tape_A = tape.copy()
            tape_A.write('A')
            tape_A.move('R')
            next_configs.append(('q_count_zeros', tape_A))
            tape_B = tape.copy()
            tape_B.write('B')
            tape_B.move('R')
            next_configs.append(('q_count_zeros', tape_B))

        elif state == 'q_count_zeros':
            if symbol == '0':
                new_tape = tape.copy()
                new_tape.move('R')
                next_configs.append(('q_count_zeros', new_tape))
            elif symbol in {'1', ' '}:
                new_tape = tape.copy()
                new_tape.move('R')
                next_configs.append(('q_find_number', new_tape))

        elif state == 'q_find_number':
            if symbol == '1':
                new_tape = tape.copy()
                new_tape.position -= 1
                next_configs.append(('q_assign', new_tape))
            elif symbol == '0':
                new_tape = tape.copy()
                new_tape.move('R')
                next_configs.append(('q_find_number', new_tape))
            elif symbol == ' ':
                new_tape = tape.copy()
                new_tape.position = 0
                next_configs.append(('q_compare', new_tape))

        elif state == 'q_compare':
            new_tape = tape.copy()
            next_configs.append(('q_accept', new_tape))

        return next_configs

    def compute_sums(self, tape_str: str) -> Tuple[int, int]:
        sum_a, sum_b = 0, 0
        i = 0
        while i < len(tape_str):
            if tape_str[i] in {'A', 'B'}:
                marker = tape_str[i]
                i += 1  
                if i < len(tape_str) and tape_str[i] == '1':
                    i += 1  
                    
                    count = 0
                    while i < len(tape_str) and tape_str[i] == '0':
                        count += 1
                        i += 1
                    num = count  
                    if marker == 'A':
                        sum_a += num
                    else:
                        sum_b += num
                else:
                    i += 1
            else:
                i += 1
        return sum_a, sum_b

    def simulate(self) -> bool:
        queue = deque([(self.state, self.tape)])
        self.seen_configurations = set()

        while queue:
            state, tape = queue.popleft()
            config = (state, str(tape), tape.position)
            if config in self.seen_configurations:
                continue
            self.seen_configurations.add(config)

            if state == 'q_accept':
                tape_str = str(tape)
                sum_a, sum_b = self.compute_sums(tape_str)
                if sum_a == sum_b:
                    return True
                continue
            if state == 'q_reject':
                continue

            next_configs = self.step(state, tape)
            queue.extend(next_configs)

        return False

    def run(self, input_str: str) -> bool:
        try:
            self.initialize_tape(input_str)
            return self.simulate()
        except Exception as e:
            print(f"Помилка: {e}")
            return False
        
    def convert_to_binary(numbers: List[int]) -> str:
        return ' '.join(format(num, 'b') for num in numbers)  



numbers = [1, 2, 6, 1, 2]
possible, subset1, subset2 = can_partition(numbers)

if possible:
    print(f"Розбиття можливе!")
    print(f"Перша підмножина: {subset1}, сума: {sum(subset1)}")
    print(f"Друга підмножина: {subset2}, сума: {sum(subset2)}")
else:
    print("Розбиття неможливе.")

Розбиття можливе!
Перша підмножина: [1, 2, 1, 2], сума: 6
Друга підмножина: [6], сума: 6
